In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [37]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
# new_dataset = "lawful-good-project/ipc_decisions_4k"
new_dataset = "lotusbro/ipc_decisions_summarized"

# Fine-tuned model
new_model = "llama-2-7b-chat-law"

# Load Model

In [17]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [12]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Load Dataset

In [16]:
dataset = load_dataset(new_dataset, split="train")
print(len(dataset), dataset[1])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

4128 {'text': 'Суд определил, что заявленное обозначение является сходным до степени смешения с противопоставленным товарным знаком, поскольку они имеют сходства в фонетическом и семантическом отношении. Также было установлено, что товары, заявленные на регистрацию в качестве товарного знака, являются однородными с товарами, против которых зарегистрирован противопоставленный товарный знак. В результате суд пришел к выводу, что заявленное обозначение не может быть зарегистрировано в качестве товарного знака в отношении товаров 30-го класса, поскольку оно не соответствует требованиям подпункта 2 пункта 6 статьи 1483 X X. Таким образом, довод заявителя о том, что решение суда от 23.10.2017 не соответствует положениям пункта 6 статьи 1483 X X, поскольку заявленное им на регистрацию в качестве товарного знака обозначение не является сходным до степени смешения в отношении однородных товаров 30-го класса X, с противопоставленным товарным знаком, признается несостоятельным.'}


# Train

In [22]:
# Load LoRA configuration
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [23]:
# Set training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [24]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4128 [00:00<?, ? examples/s]

In [25]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.624400
50,1.413200
75,1.218600
100,1.215200
125,1.179300
150,1.076300
175,1.159400
200,1.095600
225,1.126000
250,1.120200


TrainOutput(global_step=1032, training_loss=1.028080513310987, metrics={'train_runtime': 2813.8081, 'train_samples_per_second': 1.467, 'train_steps_per_second': 0.367, 'total_flos': 6.124985273165414e+16, 'train_loss': 1.028080513310987, 'epoch': 1.0})

# Save new model

In [27]:
# Save trained model
trainer.model.save_pretrained("adapter")

# Reload model in FP16 and merge it with LoRA weights
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(load_model, "adapter")
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save merged model
model.save_pretrained("model_merged")
tokenizer.save_pretrained("model_merged")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

('model_merged/tokenizer_config.json',
 'model_merged/special_tokens_map.json',
 'model_merged/tokenizer.json')

# Test new model

In [38]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
pipe = pipeline(task="text-generation", model="model_merged", torch_dtype=torch.float16, device_map="auto", max_length=1024)
pipe_old = pipeline(task="text-generation", model=base_model, torch_dtype=torch.float16, device_map="auto", max_length=1024)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
# Make chat template with context prompts

question = "Сотрудник библиотеки А. настроил полнотекстовый поиск по базе данных библиотеки и предоставил посетителям библиотеки возможность поиска и скачивания фрагментов нужных книг за пределами библиотеки. Являются ли правомерными действия А.?"

chat = [
    {"role": "user", "content": "Мне нужна юридическая консультация"},
    {"role": "assistant", "content": "Добрый день! Опишите ситуацию, чтобы я мог подсказать вам, как поступить."},
    {"role": "user", "content": question},
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False)
print(prompt)

<s>[INST] Мне нужна юридическая консультация [/INST] Добрый день! Опишите ситуацию, чтобы я мог подсказать вам, как поступить. </s><s>[INST] Сотрудник библиотеки А. настроил полнотекстовый поиск по базе данных библиотеки и предоставил посетителям библиотеки возможность поиска и скачивания фрагментов нужных книг за пределами библиотеки. Являются ли правомерными действия А.? [/INST]


In [40]:
result = pipe_old(prompt)
print(result[0]['generated_text'].replace(prompt, ""))

  К сожалению, без дополнительной информации о конкретной ситуации и базе данных, в которой работает библиотека, невозможно точно определить, правомерны или нет действия сотрудника библиотеки А.

В общем случае, библиотекари и другие сотрудники библиотек должны соблюдать законы и правила, регулирующие использование и распространение литературных произведений. В России, например, закон "Об авторском праве" устанавливает определенные права authors на их произведения, включая право на распространение и использование их произведений.

Если сотрудник библиотеки А. настроил полнотекстовый поиск по базе данных библиотеки и предоставил посетителям библиотеки возможность поиска и скачивания фрагментов нужных книг за пределами библиотеки, то это может быть расценено как нарушение авторских прав authors.

Кроме того, необходимо учитывать, что библиотеки обычно имеют договоры о лицензировании с издательствами и авторами, которые определяют, какие произведения могут быть доступны для использования 

In [41]:
result = pipe(prompt)
print(result[0]['generated_text'].replace(prompt, ""))

  Действия сотрудника библиотеки А. могут быть рассмотрены как неправомерные, поскольку он предоставил посетителям библиотеки возможность поиска и скачивания фрагментов книг за пределами библиотеки, что противоречит закону и правилам библиотечной деятельности. Библиотека является учреждением культуры и образования, и ее основной целью является сохранение и распространение книжной и информационной культуры. В рамках своей деятельности библиотека может предоставлять услуги по поиску и скачиванию книг, но только в пределах библиотеки. Предоставление возможности поиска и скачивания фрагментов книг за пределами библиотеки может привести к нарушению авторских прав и прав на использование материалов, а также к недобросовестному использованию библиотечной деятельности. Кроме того, это может привести к недобросовестному использованию библиотечной деятельности. В связи с этим, действия сотрудника библиотеки А. могут быть рассмотрены как неправомерные и могут быть оценены как нарушение закона и п

# Push to Hub

In [48]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [50]:
model.push_to_hub(new_model, use_temp_dir=True)
tokenizer.push_to_hub(new_model, use_temp_dir=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:527: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lotusbro/llama-2-7b-chat-law/commit/65dd539ba6f8812d2432f5b17bbb1d2fed6cb94d', commit_message='Upload tokenizer', commit_description='', oid='65dd539ba6f8812d2432f5b17bbb1d2fed6cb94d', pr_url=None, pr_revision=None, pr_num=None)